In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-arab/validation_data.jsonl
/kaggle/input/arabic-text-summarization-30-000/wikiHow.csv
/kaggle/input/nlpara/labeled_validation_dataset.jsonl
/kaggle/input/nlpara/arabic_train.jsonl
/kaggle/input/nlpara/decay_model_weights_016.h5
/kaggle/input/nlpara/arabic_val.jsonl
/kaggle/input/nlpara/arabic_test.jsonl


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_scheduler
from datasets import load_metric

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# sns.set(style='whitegrid', palette='muted', font_scale=1.2)
# rcParams['figure.figsize'] = 16, 10

In [4]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")



/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
# Dealing with JSON files & CSV files & Data frames
path_input = '/kaggle/input/nlpara/labeled_validation_dataset.jsonl'

# First reading JSON file and converting it to data frame
data = pd.read_json(path_input, lines=True)
data['summary'] = data['summary'].replace(r'\n', '', regex = True)

# Second reading CSV file and converting it to data frame
data = pd.read_csv(path_input)

# Third to create a CSV file from a data frame
data.to_csv('data.csv', index=False)

# Fourth to create a JSON file from a data frame
data.to_json("predictions.jsonl", lines=True, orient='records', force_ascii=False)


data.head()

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [6]:
#Data Preprocessing
    #First Checking 'duplicated rows' and removing them if they are exist
dupl = data[data.duplicated()]
if len(dupl)>0:
    data=data.drop_duplicates()
    print(len(dupl))
    
# !pip install nltk
import re
import nltk
### from nltk.stem.isri import ISRIStemmer
### st = ISRIStemmer()

# Data cleaning
def clean_text(text):
    # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A \u0660-\u0669 0-9 \s ( ) : . ، ؛ ]+','',text)
    
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\.{2,}', '', text)

    return text

# Data preprocessing
def pre_process(text):
    tokens = nltk.word_tokenize(text)
    stop_words = ["و", "في", "من", "على", "إلى", "عن", "فيه", "عليه", "هو", "هي"]
    text = " ".join([word for word in tokens if word not in stop_words])

    return text

data['paragraph'] = data['paragraph'].apply(clean_text)

print(data['paragraph'])

# Apply the clean_text function to the content column
data['paragraph'] = data['paragraph'].apply(clean_text)
print(data['paragraph'])


0      وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...
1      ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...
2      قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...
3      دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...
4      السوريون يصرون على استقلال بلادهم و مثلما رفض ...
                             ...                        
149    حزب الوفد سيحتفل بمئوية ثورة 99 يوم 9 مارس الق...
150    حيث أعلن مجلس قيادة الثورة في يونيه 9 قيام الج...
151    وبرغم أن عبد الرحمن فهمي كان يضم في ذلك الجهاز...
152    ولم تقتصر مقومات بورسعيد كمدينة عالمية منذ نشأ...
153    أول رئيس للجزائر بعد الاستقلال الرئيس أحمد بن ...
Name: paragraph, Length: 154, dtype: object


In [7]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
preprocessed_text=[]
def preprocess_arabic_Remove_stop(text):
    
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Stem the words
    #words = [stemmer.stem(word) for word in words]
    
    # Join the words back into a single string
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text

data['paragraph'] = data['paragraph'].apply(preprocess_arabic_Remove_stop)
data['paragraph'] = data['paragraph'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,example_id,paragraph,summary
0,0,وتحت عنوان الكارثة التحدى يبدأ الكاتب عرض الكت...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو الدولتين بالعريضة وقعها ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب اعلان الجنرال الفرنسي هنري غورو...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه اي دوله وليست اي شعب لابل انه...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون استقلال بلادهم مثلما رفض السوري...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [8]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=3bbc5493954d2d1b24a5796c2abacdc6abaa6aa503972eee74dc34c690e0a240
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data=data,
        tokenizer=tokenizer,
        text_max_token_len = 800,
        summary_max_token_len = 150
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['paragraph']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [10]:
df_train, df_test = train_test_split(data, test_size=0.2, random_state=42)

train_dataset = SummaryDataset(data=df_train)
test_dataset = SummaryDataset(data=df_test)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)
eval_dataloader = DataLoader(test_dataset, batch_size=2)

In [11]:
num_epochs = 4

num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         print(predictions)
#         print(batch["labels"])
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5-Arabic.pth')

    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/248 [00:00<?, ?it/s]

epoch: 1 -- loss: 2.4432668685913086
epoch: 2 -- loss: 2.479388952255249
epoch: 3 -- loss: 1.5066146850585938
epoch: 4 -- loss: 1.2335790395736694


In [12]:
metric= load_metric("rouge")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'rouge1': AggregateScore(low=Score(precision=0.6281559139784947, recall=0.6281559139784947, fmeasure=0.6281559139784947), mid=Score(precision=0.6579569892473118, recall=0.6579569892473118, fmeasure=0.6579569892473118), high=Score(precision=0.6851666666666667, recall=0.6851666666666667, fmeasure=0.6851666666666667)),
 'rouge2': AggregateScore(low=Score(precision=0.31846178826585836, recall=0.31846178826585836, fmeasure=0.31846178826585836), mid=Score(precision=0.35722017752760327, recall=0.35722017752760327, fmeasure=0.35722017752760327), high=Score(precision=0.39424659017103275, recall=0.39424659017103275, fmeasure=0.39424659017103275)),
 'rougeL': AggregateScore(low=Score(precision=0.5113763440860214, recall=0.5113763440860214, fmeasure=0.5113763440860214), mid=Score(precision=0.5473118279569891, recall=0.5473118279569891, fmeasure=0.5473118279569891), high=Score(precision=0.5776344086021504, recall=0.5776344086021504, fmeasure=0.5776344086021504)),
 'rougeLsum': AggregateScore(low=S

In [13]:
def summarizeText(text, model=model):
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'].to(device),
        attention_mask=text_encoding['attention_mask'].to(device),
        max_length=150,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )    

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)


In [14]:
path_input = '/kaggle/input/data-arab/validation_data.jsonl'

# Read the JSONL file
df = pd.read_json(path_input, lines=True)

df.head()

,example_id,paragraph
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...


In [15]:
#Data Preprocessing
    #First Checking 'duplicated rows' and removing them if they are exist
dupl = df[df.duplicated()]
if len(dupl)>0:
    df=df.drop_duplicates()
    print(len(dupl))
    
    #Second Checking 'empty cells' and removing them if they are exist
    #Here we deal with empty cells (either removing them Or setting them with default data) but we won't do that because data is cleaned

    #Third Cleaning data from (Arabic and English digits, special characters, and extra spaces)
import re
def clean_text(text):

        # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A\u0660 - \0669\s]+', '', text)

        # Remove "English digits"
    text = re.sub('\d+', '', text)

        # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply the clean_text function to the content column
df['paragraph'] = df['paragraph'].apply(clean_text)
df['paragraph']

0      وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1      الفتوحات الإسلامية في عصر الخلفاء الراشدين وبع...
2      فتوحات بلاد شمال إفريقيا قام المسلمون بالتوجه ...
3      وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4      عزيزي الطالب عزيزتي الطالبة بعد أن توقفت حركة ...
                             ...                        
267    وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268    كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269    حشدت فرنسا قواتها وأعلنت الحرب في 9 يوليو أي ب...
270    تعتبر قناة السويس هي من أهم الطرق الملاحية الت...
271    فالثورة الفرنسية وهي محل شاهد في صراع الطبقات ...
Name: paragraph, Length: 272, dtype: object

In [16]:
import pandas as pd

# Function to apply the model to each instance
def apply_model(instance):
    # Apply your model to the instance and return the output
    output = summarizeText(instance)
    return output

# Create an empty list to store the model outputs
model_outputs = []

# Iterate over the dataframe and apply the model to each instance
for index, row in df.iterrows():
    instance = row['paragraph']  # Assuming 'text' is the column containing the input data
    output = apply_model(instance)
    model_outputs.append(output)

# Add the model outputs as a new column to the dataframe
df['summary'] = model_outputs

# Display the updated dataframe
print(df['summary'])


0      فتح بيت المقدس هو نموذجا لفتوحات بلاد الشام، و...
1      انتهى عمر بن الخطابة من خطبته لتفقد كنيسة القي...
2      قام المسلمون بالتوجه غربا لفتح شمال إفريقيا وح...
3      فتح سبيطلة يعتبر نموذجا لفتوحات بلاد شمال إفري...
4      توقفت حركة الفتوحات الإسلامية بسبب فتنة عثمان ...
                             ...                        
267    ظهرت الدراسات قصرت اهتمامها بالتاريخ من زاوية ...
268    ظلت الزراعة المصرية في عصرها الذهبي، ولما أثرى...
269    حشدت فرنسا قواتها وأرسلت قوات إلى الاتحاد والت...
270    قناة السويس هي من أهم الطرق الملاحية التي تعتم...
271    بعد الثورة الفرنسية، نجحت الحكومة في القضاء عل...
Name: summary, Length: 272, dtype: object


In [17]:
#Data Preprocessing
    #First Checking 'duplicated rows' and removing them if they are exist
dupl = data[data.duplicated()]
if len(dupl)>0:
    data=data.drop_duplicates()
    print(len(dupl))
    
    #Second Checking 'empty cells' and removing them if they are exist
    #Here we deal with empty cells (either removing them Or setting them with default data) but we won't do that because data is cleaned

    #Third Cleaning data from (Arabic and English digits, special characters, and extra spaces)
import re
def clean_text(text):

        # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A\u0660 - \0669\s]+', '', text)

        # Remove "English digits"
    ##text = re.sub('\d+', '', text)

        # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply the clean_text function to the content column
df['summary'] = df['summary'].apply(clean_text)
print(df['summary'])

0      يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1      دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2      أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3      مصر هي أم البلاد وقائدة العرب فهي أرض بلاد الن...
4      الشعب السوري يصر على استقلال بلدهم من السيطرة ...
                             ...                        
149    احتفال مئوية ثورة 99 كان من منطلق وطني ليس حزب...
150    مجلس قيادة الثورة أعلن عن قيام الجمهورية المصر...
151    ضم عبد الرحمن فهمي في الجهاز السري عدد كبير من...
152    امتدت بورسعيد لكي تشمل الطابع الثقافي للمدينة ...
153    كان أحمد بن بيلا أول رئيس للجزائر بعد الاستقلا...
Name: summary, Length: 154, dtype: object


In [18]:
df2 = df.drop(columns='paragraph')
# Assuming you already have a dataframe named "df"

# Convert dataframe to JSON
df2.to_json("predictions.jsonl", lines=True, orient='records', force_ascii=False)